In [28]:
import pandas as pd
import numpy as np
import json

class MeanRevStrategy:
    def __init__(self, price_data):
        self.price_data = price_data  # DataFrame with timestamp indexed asset prices
        self.leverage = 50
        self.initial_portfolio_value = 2000

    def calculate_zscore(self, spread_series, window):
        mean = spread_series.rolling(window=window).mean()
        std = spread_series.rolling(window=window).std()
        z_score = (spread_series - mean) / std
        return z_score

    def simulate_trade(self, market, POSITION_SIZE, Z_SCORE, window):
        spread_series = self.price_data[market]
        z_score_series = self.calculate_zscore(spread_series, window)

        # Initialize portfolio values tracking
        portfolio_value = self.initial_portfolio_value
        in_position = False
        position_type = None  # Track whether the position is long or short

        for idx in range(len(self.price_data)):
            current_z_score = z_score_series.iloc[idx]
            current_price = spread_series.iloc[idx]
            trade_return = 0.0

            # Flipping positions based on Z_SCORE threshold
            if in_position:
                # Calculate return based on price change since entry
                price_increase = (current_price - entry_price) * ((POSITION_SIZE * self.leverage) / entry_price)
                if position_type == "long" and current_z_score >= Z_SCORE:
                    trade_return = price_increase - (POSITION_SIZE * self.leverage) * 0.02
                    in_position, position_type, entry_price = False, None, None
            elif not in_position:
                if current_z_score <= -Z_SCORE:  # Enter long position
                    in_position, position_type, entry_price = True, "long", current_price

            portfolio_value += trade_return

        # Exiting the last open position at the end of the data
        if in_position:
            # Simulate exiting the position with no additional profit or loss
            in_position, position_type = False, None

        # Returning the portfolio values for further analysis
        return portfolio_value

    def run_monte_carlo_simulation(self, iterations):
        best_parameters = {}
        highest_returns = -np.inf

        for _ in range(iterations):
            WINDOW = np.random.randint(290, 310)
            POSITION_SIZE = 1
            Z_SCORE = np.random.uniform(2.9, 3.1)
            total_returns = self.trade_all_markets(WINDOW, POSITION_SIZE, Z_SCORE)
            # Check if this simulation yielded higher returns than previous best
            if total_returns > highest_returns:
                highest_returns = total_returns
                best_parameters = {
                    'returns': highest_returns,
                    'window': WINDOW,
                    'position_size': POSITION_SIZE,
                    'z_score': Z_SCORE
                }

        # Save the best parameters to a JSON file
        with open('best_parameters.json', 'w') as f:
            json.dump(best_parameters, f, indent=4)

        return best_parameters

    def trade_all_markets(self, WINDOW, POSITION_SIZE, Z_SCORE):
        total_returns = 0.0
        for market in self.price_data.columns:
            if market == 'time':
                continue

            portfolio_value = self.simulate_trade(market, POSITION_SIZE, Z_SCORE, WINDOW)
            returns = portfolio_value - self.initial_portfolio_value
            total_returns += returns

        return total_returns


In [29]:
data_15m = pd.read_csv('data_test_15m.csv')
data_15m = data_15m.tail(2000)

strategy = MeanRevStrategy(data_15m)
best_params_15m = strategy.run_monte_carlo_simulation(iterations=20)
print(best_params_15m)

{'returns': 65.38119585037248, 'window': 304, 'position_size': 1, 'z_score': 3.0869760767056573}


In [27]:
data_1H = pd.read_csv('data_test_1H.csv')
data_1H = data_1H.tail(2000)

strategy = MeanRevStrategy(data_1H)
best_params_1H = strategy.run_monte_carlo_simulation(iterations=10)
print(best_params_1H)

{'returns': 49.085937340701776, 'window': 455, 'position_size': 1, 'z_score': 2.583233744083843}


In [24]:
data_15m = pd.read_csv('data_test_15m.csv')
data_15m = data_15m.tail(2000)

strategy = MeanRevStrategy(data_15m)
best_params_15m = strategy.run_monte_carlo_simulation(iterations=1000)
print(best_params_15m)

{'returns': 65.86947490532543, 'window': 306, 'position_size': 1, 'z_score': 3.071749251219703}


In [25]:
data_1H = pd.read_csv('data_test_1H.csv')
data_1H = data_1H.tail(2000)

strategy = MeanRevStrategy(data_1H)
best_params_1H = strategy.run_monte_carlo_simulation(iterations=1000)
print(best_params_1H)

{'returns': 46.442150018695656, 'window': 379, 'position_size': 1, 'z_score': 2.5835775902305604}
